In [6]:
import sys
from glob import glob
from collections import defaultdict

sys.path.append("/scratch/group/csce435-f23/python-3.8.17/lib/python3.8/site-packages")
sys.path.append("/scratch/group/csce435-f23/thicket")

import pandas as pd

import thicket as th

In [7]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# POINT THIS AT YOUR DATA

In [16]:
# Point this at the top directory of all your cali files for all of the implementations. Other files can be in the directory too, that is ok.
FILES_LOCATION = "."

# Reader check
Can the files be read in one-by-one

In [19]:
working_files = []
error_files = []
i = 0
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        tk = th.Thicket.from_caliperreader(file)
        working_files.append(file)
    except Exception:
        i += 1
        error_files.append(file)

In [20]:
print("Files that could be read in individually (one-by-one):")
print(f"{len(working_files)}/{len(team_files)} ({len(working_files)/len(team_files)*100}%)")

Files that could be read in individually (one-by-one):
16/16 (100.0%)


# Check for Metadata columns

Check for the necessary metadata columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3b-collect-metadata)

In [21]:
team_metadata_valid = []
metadata_columns=['cali.caliper.version', 'spot.options', 'spot.channels', 'cali.channel',
    'launchdate',
    'libraries', 'cmdline', 'cluster', 'Algorithm', 'ProgrammingModel',
    'Datatype', 'SizeOfDatatype', 'InputSize', 'InputType',
    'group_num', 'implementation_source']
mpi_cols = ['num_procs']
cuda_cols = ['num_threads', 'num_blocks',]
metadata_col_dict = defaultdict(lambda: [])

team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        valid = True
        tk = th.Thicket.from_caliperreader(file)
        cols = tk.metadata.columns
        model_to_check = []
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            model_to_check = metadata_columns + cuda_cols
        else:
            model_to_check = metadata_columns + mpi_cols
        for col in model_to_check:
            if col not in cols:
                metadata_col_dict[list(tk.profile_mapping.values())[0]].append(col)
                valid=False
        if valid:
            team_metadata_valid.append(file)
    except KeyError:
        pass

for file, cols in metadata_col_dict.items():
    print(f"File '{file}' missing metadata columns:\n\t{cols}")

# Check for DataFrame columns

Check for the necessary DataFrame columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#4c-you-should-measure-the-following-performance-metrics). For the GPU columns, you need one or the other column in the tuple, not both.

In [22]:
team_dataframe_valid = []
necessary_columns = ["Min time/rank","Max time/rank","Avg time/rank","Total time",]
not_gpu_columns = ["Variance time/rank",]
gpu_columns=[("Avg GPU time/rank", "Avg GPU Time/rank"),
             ("Min GPU time/rank", "Min GPU Time/rank"),
             ("Max GPU time/rank", "Max GPU Time/rank"),
             ("Total GPU time", "Total GPU Time"),]
def check_df_cols(tk, dict):
    valid = True
    cols = tk.dataframe.columns
    for col in necessary_columns:
        if col not in cols:
            dict[list(tk.profile_mapping.values())[0]].append(col)
            valid = False
    if "ProgrammingModel" in tk.metadata.columns:
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            for col in gpu_columns:
                if col[0] not in cols and col[1] not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
        else:
            for col in not_gpu_columns:
                if col not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
    return valid

dataframe_col_dict = defaultdict(lambda: [])
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    tk = th.Thicket.from_caliperreader(file)
    valid = check_df_cols(tk, dataframe_col_dict)
    if valid:
        team_dataframe_valid.append(file)

for file, cols in dataframe_col_dict.items():
    print(f"File '{file}' missing dataframe columns: {cols}")

# Try all files together

In [23]:
tk = th.Thicket.from_caliperreader(team_files)

# Check tree

Should be no different from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3a-caliper-instrumentation), spelling and all.

In [24]:
tk.statsframe.dataframe["time"] = 1
print(tk.tree())

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2023.3.0

1.000 main
├─ 1.000 comm
│  └─ 1.000 comm_large
│     ├─ 1.000 MPI_Bcast
│     └─ 1.000 MPI_Gather
├─ 1.000 comp
│  └─ 1.000 comp_large
├─ 1.000 correctness_check
└─ 1.000 data_init

Legend (Metric: time Min: 1.00 Max: 1.00)
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00

name User code    ◀  Only in left graph    ▶  Only in right graph



In [25]:
# Groupby programming model. Should result in 2 thickets, MPI and CUDA.
gb_pmodel = tk.groupby("ProgrammingModel")

1  thickets created...
{'MPI': <thicket.thicket.Thicket object at 0x2aef772c1040>}


In [28]:
# Groupby the parameters we ran with. After this operation, each Thicket in gb_total should contain profiles with unique InputSizes (there should be no duplicate input sizes).
# gb_cuda = gb_pmodel["CUDA"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_threads"])
gb_mpi = gb_pmodel["MPI"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_procs"])
#gb_total = {**gb_cuda, **gb_mpi}
gb_total = {**gb_mpi}

4  thickets created...
{('MPI', 'EnumerationSort', '1%%perturbed', 1024): <thicket.thicket.Thicket object at 0x2aef76f72bb0>, ('MPI', 'EnumerationSort', 'Random', 1024): <thicket.thicket.Thicket object at 0x2aef77289820>, ('MPI', 'EnumerationSort', 'ReverseSorted', 1024): <thicket.thicket.Thicket object at 0x2aef7724a880>, ('MPI', 'EnumerationSort', 'Sorted', 1024): <thicket.thicket.Thicket object at 0x2aef770fd850>}


In [29]:
# Compose all of the data back together. If this step errors, you probably have duplicate inputsizes. Run 1a to check for this.
ctk = th.Thicket.concat_thickets(
    thickets=list(gb_total.values()),
    axis="columns",
    headers=list(gb_total.keys()),
    metadata_key="InputSize"
)

In [30]:
ctk.dataframe.head(50)

(MPI, EnumerationSort, 1%%perturbed, 1024)  \
                                                                                                   nid   
node                                              InputSize                                              
{'name': 'main', 'type': 'function'}              65536                                              1   
                                                  262144                                             1   
                                                  1048576                                            1   
                                                  4194304                                            1   
{'name': 'comm', 'type': 'function'}              65536                                              3   
                                                  262144                                             3   
                                                  1048576                                            3   
                                                  4194304                                            3   
{'name': 'comm_large', 'type': 'function'}        65536                                              4   
                                                  262144                                             4   
                                                  1048576                                            4   
                                                  4194304                                            4   
{'name': 'MPI_Bcast', 'type': 'function'}         65536                                              5   
                                                  262144                                             5   
                                                  1048576                                            5   
                                                  4194304                                            5   
{'name': 'MPI_Gather', 'type': 'function'}        65536                                              8   
                                                  262144                                             8   
                                                  1048576                                            8   
                                                  4194304                                            8   
{'name': 'comp', 'type': 'function'}              65536                                              6   
                                                  262144                                             6   
                                                  1048576                                            6   
                                                  4194304                                            6   
{'name': 'comp_large', 'type': 'function'}        65536                                              7   
                                                  262144                                             7   
                                                  1048576                                            7   
                                                  4194304                                            7   
{'name': 'correctness_check', 'type': 'function'} 65536                                              9   
                                                  262144                                             9   
                                                  1048576                                            9   
                                                  4194304                                            9   
{'name': 'data_init', 'type': 'function'}         65536                                              2   
                                                  262144                                             2   
                                                  1048576                                            2   
                                

# 1A

Check for duplicate input sizes

In [31]:
i = 0
for key in list(gb_total.keys()):
    print(i)
    print(gb_total[key].profile_mapping)
    print(gb_total[key].metadata["InputSize"])
    i += 1

0
OrderedDict([(296084103, './s3-p1024-a1048576.cali'), (1384767298, './s3-p1024-a262144.cali'), (2812938463, './s3-p1024-a4194304.cali'), (3782295928, './s3-p1024-a65536.cali')])
profile
296084103     1048576
1384767298     262144
2812938463    4194304
3782295928      65536
Name: InputSize, dtype: int64
1
OrderedDict([(346302474, './s0-p1024-a262144.cali'), (1439536646, './s0-p1024-a1048576.cali'), (1693053521, './s0-p1024-a4194304.cali'), (1928734459, './s0-p1024-a65536.cali')])
profile
346302474      262144
1439536646    1048576
1693053521    4194304
1928734459      65536
Name: InputSize, dtype: int64
2
OrderedDict([(391301741, './s2-p1024-a262144.cali'), (2587501010, './s2-p1024-a4194304.cali'), (2725407198, './s2-p1024-a1048576.cali'), (4170353367, './s2-p1024-a65536.cali')])
profile
391301741      262144
2587501010    4194304
2725407198    1048576
4170353367      65536
Name: InputSize, dtype: int64
3
OrderedDict([(619903883, './s1-p1024-a65536.cali'), (651100667, './s1-p1024-a419